# Перегрузка операторов в `C++`

**Перегрузка операторов** — это механизм, позволяющий использовать привычные математические и логические операторы для работы с пользовательскими типами данных. Это делает код более читаемым и понятным, а также позволяет использовать уже знакомые операции для работы с новыми типами данных.

Одним из примеров перегрузки операторов является оператор +=, который используется для сложения двух переменных. В контексте перегрузки операторов, оператор += может быть переопределён для работы с пользовательскими типами данных.

В данной лекции разберем механизм перегрузки оператора на пример класса рациональных числе(дробных). Пример взят с сайта [нашей кафедры](https://cmp.phys.msu.su/sites/default/files/Seminar21.pdf).


## Класс рациональных чисел

Добавим библиотеки:
```cpp
#include <iostream>
using namespace std;
```
Начнем писать наше класс дробных чисел:
```cpp
class Ratio {
    private:
        int a,b;
    public:
        explicit Ratio(int na=0, int nb=0) : a(na), b(nb) //Конструктор
        { 
            cout << "Hi: (" << a << "/" << b << ")" << endl; 
        }
        Ratio(const Ratio& src) : a(src.a), b(src.b) // Конструктор копирования
        { 
            cout << "Hi: (" << a << "/" << b << ")" << endl << flush; 
        }
        ~Ratio() // Деструктор
        { 
            cout << "RIP: (" << a << "/" << b << ")" << endl << flush; 
        }

```

## Перегрузка оператора присваивания `=` :
```cpp
Ratio& operator=( const Ratio& src ) 
{ 
    a=src.a; 
    b=src.b; 
    return *this; 
}
```
В методах класса мы можем использовать ключевое слово `this`, который является указателем на объект класса, от которого он был вызван.

`*this` --- разыменованный указатель, т.е. мы возвращаем объект класса.

В данном примере, мы делаем возврат класса, для того, чтобы у нас работало множественное присваивание:
```cpp
Ratio r1(1,2), r2(3,7), r3(4, 9);

r3 = r1 = r2;
```
```{note}
Чему будут равны r1, r2, r3 в примере выше?
```

## Перегрузка оператора присваивания `+=` :
```cpp
 Ratio& operator+=( const Ratio& src )
    { 
        a = a * src.b + b * src.a;
        b *= src.b;
        return *this;
    }

```

## Вспомогательные методы и перегрузка оператора `+` :
Напишем константные методы доступа:
```cpp
int chislitel() const { return a; }
int znamenatel() const { return b; }
```
Методы где после прототипа функции, но до ее тела написано ключевое слово `const` --- называются константными.
Их использование необходимо при вызове через **константные** ссылки объекта класса. Звучит и правда сложно, но более подробно зачем и почему будет описано ниже при описании перегрузки оператора `<<`.

Операция переворота дроби:
```cpp
Ratio& reverse() { int tmp=a; a=b; b=tmp; return *this; }
```

Явное поведение класса при преобразовании `int`:
```cpp
operator int() const { return a/b; }
```

```cpp
 Ratio operator+( const Ratio& src2 ) const
 { 
    Ratio res(*this); // создаем объект класса с помощью конструктора копирования
    res += src2;
    return res;
 }
```

Все методы выше описывают класс рациональных чисел. Дальнейшие Перегрузки операторов производятся вне тела класса.


## Перегрузка оператора `<<` :

Рассмотрим как объявить программе вывод объекта класса с помощью `cout`
```cpp
ostream& operator<<(ostream& os, const Ratio& src)
{ 
    os << "(" << src.chislitel() << "/";
    os << src.znamenatel() << ")" << flush;
    return os;
}
```
`ostream` — это класс в языке C++, который представляет потоки вывода. Он позволяет отправлять данные на устройства вывода, такие как консоль или файлы.

Объект типа `ostream` получает значения различных типов, преобразует их в последовательность символов и передаёт через буфер в определённое место для вывода.

Тут и раскрывается смысл константных методов. Как уже было сказано, константные ссылки были введены в параметры функции, дабы ограничить непреднамеренное изменение передаваемого объекта. Но эта защита, так же ограничивает нас в использовании методов класса. Т.е. если мы константные методы `chislitel()` и `znamenatel()`  превратим в обычные, то при вызове:
```cpp
Ratio r1;
cout << r; 
```
у нас выведется ошибка.